In [14]:
import torch
import torch.nn as nn
from ultralytics.nn.tasks import DetectionModel
from ultralytics.utils.torch_utils import initialize_weights

class MultiModalYOLO(DetectionModel):
    def __init__(self, cfg="yolov8n.yaml"):
        # Najpierw inicjalizujemy DetectionModel
        super().__init__(cfg)
        
        # Zachowaj oryginalny backbone dla RGB
        self.backbone_rgb = self.model.backbone
        
        # Stwórz nowy backbone dla IR z modyfikacjami
        self.backbone_ir = self._create_ir_backbone()
        
        # Warstwy fuzji
        self.fusion_convs = nn.ModuleList([
            nn.Conv2d(2 * ch, ch, kernel_size=1) for ch in self.backbone_rgb.out_channels
        ])

    def _create_ir_backbone(self):
        # Klonujemy architekturę backbone'u
        backbone_ir = self.model.backbone.__class__()
        
        # Modyfikujemy pierwszą warstwę Conv2d na 1 kanał wejściowy
        first_conv = None
        for name, module in backbone_ir.named_children():
            if isinstance(module, nn.Conv2d):
                first_conv = module
                break
                
        if first_conv:
            new_conv = nn.Conv2d(
                in_channels=1,
                out_channels=first_conv.out_channels,
                kernel_size=first_conv.kernel_size,
                stride=first_conv.stride,
                padding=first_conv.padding,
                bias=first_conv.bias is not None
            )
            initialize_weights(new_conv)
            backbone_ir._modules[name] = new_conv
            
        return backbone_ir

    def forward(self, x_rgb, x_ir=None):
        # Obsługa przypadku inicjalizacji przez parent class
        if x_ir is None:
            x_ir = torch.zeros_like(x_rgb[:, :1])  # Zachowaj batch i rozdzielczość
            
        # Ekstrakcja cech
        p3_rgb, p4_rgb, p5_rgb = self.backbone_rgb(x_rgb)
        p3_ir, p4_ir, p5_ir = self.backbone_ir(x_ir)
        
        # Fuzja
        p3 = self.fusion_convs[0](torch.cat([p3_rgb, p3_ir], dim=1))
        p4 = self.fusion_convs[1](torch.cat([p4_rgb, p4_ir], dim=1))
        p5 = self.fusion_convs[2](torch.cat([p5_rgb, p5_ir], dim=1))
        
        return self.model.head(self.model.neck([p3, p4, p5]))

In [15]:
import torch
from ultralytics import YOLO

# Konfiguracja
device = "cuda" if torch.cuda.is_available() else "cpu"

# Inicjalizacja modelu
model = MultiModalYOLO().to(device)

# Test forward pass z przykładowymi danymi
x_rgb = torch.randn(1, 3, 640, 640).to(device)  # RGB (3 kanały)
x_ir = torch.randn(1, 1, 640, 640).to(device)   # IR (1 kanał)

# Test 1: Zwykły forward
outputs = model(x_rgb, x_ir)
print([t.shape for t in outputs])

# Test 2: Symulacja inicjalizacji przez DetectionModel (tylko 1 tensor)
dummy_input = torch.randn(1, 3, 640, 640).to(device)  # Na potrzeby inicjalizacji
try:
    model(dummy_input)  # Wykorzysta wewnętrznie x_ir = zeros
    print("Inicjalizacja udana!")
except Exception as e:
    print(f"Błąd: {e}")


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

AttributeError: 'MultiModalYOLO' object has no attribute 'backbone_rgb'